In [1]:
!pip install ../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master/ > /dev/null # no output
!pip install torch_optimizer --no-index --find-links=file:///kaggle/input/torch-optimizer/torch_optimizer

  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
Looking in links: file:///kaggle/input/torch-optimizer/torch_optimizer
Processing /kaggle/input/torch-optimizer/torch_optimizer/torch_optimizer-0.1.0-py3-none-any.whl
Processing /kaggle/input/torch-optimizer/torch_optimizer/pytorch_ranger-0.1.1-py3-none-any.whl


In [2]:
import os
import gc
gc.enable()
import sys
import math
import json
import time
import random
from glob import glob
from datetime import datetime

import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import multiprocessing
from sklearn.preprocessing import LabelEncoder

import torch
import torchvision
from torch import Tensor
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.nn.parameter import Parameter
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler
from tqdm import tqdm

import efficientnet_pytorch

import torch_optimizer as optim
import albumentations as A

import sklearn

import warnings
warnings.filterwarnings("ignore")

In [3]:
IN_KERNEL = os.environ.get('KAGGLE_WORKING_DIR') is not None
BATCH_SIZE = 64
NUM_WORKERS = multiprocessing.cpu_count()
NUM_TOP_PREDICTS = 1

In [4]:
test_dir = '../input/landmark-recognition-2021/test/'

In [5]:
class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe: pd.DataFrame, image_dir:str, mode: str):
        self.df = dataframe
        self.mode = mode
        self.image_dir = image_dir
        
        transforms_list = []
        if self.mode == 'train':
            # Increase image size from (64,64) to higher resolution,
            # Make sure to change in RandomResizedCrop as well.
            transforms_list = [
                transforms.Resize((64,64)),
                transforms.RandomHorizontalFlip(),
                transforms.RandomChoice([
                    transforms.RandomResizedCrop(64),
                    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
                    transforms.RandomAffine(degrees=15, translate=(0.2, 0.2),
                                            scale=(0.8, 1.2), shear=15,
                                            resample=Image.BILINEAR)
                ]),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                      std=[0.229, 0.224, 0.225]),
            ]
        else:
            transforms_list.extend([
                # Keep this resize same as train
                transforms.Resize((64,64)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                      std=[0.229, 0.224, 0.225]),
            ])
        self.transforms = transforms.Compose(transforms_list)

    def __getitem__(self, index: int):
        image_id = self.df.iloc[index].id
        image_path = f"{self.image_dir}/{image_id[0]}/{image_id[1]}/{image_id[2]}/{image_id}.jpg"
        image = Image.open(image_path)
        image = self.transforms(image)

        if self.mode == 'test':
            return {'image':image}
        else:
            return {'image':image, 
                    'target':self.df.iloc[index].landmark_id}

    def __len__(self) -> int:
        return self.df.shape[0]

In [6]:
class EfficientNetEncoderHead(nn.Module):
    def __init__(self, depth, num_classes):
        super(EfficientNetEncoderHead, self).__init__()
        self.depth = depth
        self.base = efficientnet_pytorch.EfficientNet.from_name(f'efficientnet-b{self.depth}')#tropicbird
        pretrained_file = glob(f'../input/efficientnet-pytorch/efficientnet-b{self.depth}*')[0]
        checkpoint = torch.load(pretrained_file)
        self.base.load_state_dict(checkpoint)
        
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.output_filter = self.base._fc.in_features
        self.classifier = nn.Linear(self.output_filter, num_classes)
    def forward(self, x):
        x = self.base.extract_features(x)
        x = self.avg_pool(x).squeeze(-1).squeeze(-1)
        x = self.classifier(x)
        return x

In [7]:
def inference(data_loader, model):
    model.eval()

    activation = nn.Softmax(dim=1)
    all_predicts, all_confs, all_targets = [], [], []

    with torch.no_grad():
        for i, data in enumerate(tqdm(data_loader, disable=IN_KERNEL)):
            if data_loader.dataset.mode != 'test':
                input_, target = data['image'], data['target']
            else:
                input_, target = data['image'], None

            output = model(input_.cuda())
            output = activation(output)

            confs, predicts = torch.topk(output, NUM_TOP_PREDICTS)
            all_confs.append(confs)
            all_predicts.append(predicts)

            if target is not None:
                all_targets.append(target)

    predicts = torch.cat(all_predicts)
    confs = torch.cat(all_confs)
    targets = torch.cat(all_targets) if len(all_targets) else None

    return predicts, confs, targets

In [8]:
def generate_submission(test_loader, model, label_encoder):
    predicts_gpu, confs_gpu, _ = inference(test_loader, model)
    predicts, confs = predicts_gpu.cpu().numpy(), confs_gpu.cpu().numpy()

    #The modified labels are inversed to the original labels
    labels = [label_encoder.inverse_transform(pred) for pred in predicts]
    print('labels')
    print(np.array(labels))
    print('confs')
    print(np.array(confs))

    sub = test_loader.dataset.df

    def concat(label: np.ndarray, conf: np.ndarray):
        return ' '.join([f'{str(L)} {str(np.round(c,4))}' for L, c in zip(label, conf)])
    sub['landmarks'] = [concat(label, conf) for label, conf in zip(labels, confs)]

    sub = sub.set_index('id')
    sub.to_csv('submission.csv')

In [9]:
test_filenames=[]
for dirname, _, filenames in os.walk('../input/landmark-recognition-2021/test'):
    for filename in filenames:
        test_filenames.append(filename.split(".")[0])
test=pd.DataFrame({"id":test_filenames,"landmarks":""})

In [10]:
test_dataset = ImageDataset(test, test_dir, mode='test')
test_loader = DataLoader(test_dataset, 
                         batch_size=BATCH_SIZE,
                         shuffle=False, num_workers=NUM_WORKERS)

In [11]:
label_encoder = LabelEncoder()
label_encoder.classes_ = np.load('../input/pytorch-starter-train-efficientnet/classes.npy')

In [12]:
model = EfficientNetEncoderHead(depth=0, num_classes=len(label_encoder.classes_))
model.cuda()
model.load_state_dict(torch.load("../input/pytorch-starter-train-efficientnet/new_weight_efficientnet.pth"))

<All keys matched successfully>

In [13]:
generate_submission(test_loader, model, label_encoder)
pd.read_csv("./submission.csv")

100%|██████████| 162/162 [01:32<00:00,  1.76it/s]


labels
[[ 89939]
 [121658]
 [ 16427]
 ...
 [  7160]
 [136675]
 [ 81882]]
confs
[[0.16871728]
 [0.14075005]
 [0.15300055]
 ...
 [0.03906488]
 [0.33316883]
 [0.20020518]]


,id,landmarks
0,777f9efff0fc6b81,89939 0.1687
1,7774e44062fbd8bc,121658 0.1408
2,777173e839e6cfa7,16427 0.153
3,7726658184c7e337,166106 0.1389
4,77be72c73bbf4f18,178218 0.0855
...,...,...
10340,dd912d66515ce5f0,171140 0.0992
10341,dd90d3737bbbcc05,47516 0.0332
10342,dd6db4a564eebf38,7160 0.0391
10343,dddcd5b7d09cd416,136675 0.3332
